In [3]:
!pip install kagglehub

In [4]:
import kagglehub

# Download dataset
path = kagglehub.dataset_download("shayanfazeli/heartbeat")

print("Dataset path:", path)

Using Colab cache for faster access to the 'heartbeat' dataset.
Dataset path: /kaggle/input/heartbeat


In [5]:
import pandas as pd

train_df = pd.read_csv(path + "/mitbih_train.csv", header=None)
test_df  = pd.read_csv(path + "/mitbih_test.csv", header=None)

In [6]:
import pandas as pd

train_df = pd.read_csv("/root/.cache/kagglehub/datasets/shayanfazeli/heartbeat/versions/1/mitbih_train.csv", header=None)
test_df  = pd.read_csv("/root/.cache/kagglehub/datasets/shayanfazeli/heartbeat/versions/1/mitbih_test.csv", header=None)

In [7]:
# Basic information
print("Train shape:", train_df.shape)
print("Test shape :", test_df.shape)

# First few rows
train_df.head()

Train shape: (87554, 188)
Test shape : (21892, 188)


,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.977941,0.926471,0.681373,0.245098,0.154412,0.191176,0.151961,0.085784,0.058824,0.049020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.960114,0.863248,0.461538,0.196581,0.094017,0.125356,0.099715,0.088319,0.074074,0.082621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,0.659459,0.186486,0.070270,0.070270,0.059459,0.056757,0.043243,0.054054,0.045946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.925414,0.665746,0.541436,0.276243,0.196133,0.077348,0.071823,0.060773,0.066298,0.058011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.967136,1.000000,0.830986,0.586854,0.356808,0.248826,0.145540,0.089202,0.117371,0.150235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
train_df[187].value_counts().sort_index()

,count
187,
0.0,72471
1.0,2223
2.0,5788
3.0,641
4.0,6431


In [9]:
!pip install tensorflow

In [10]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# Split features and labels
X_train = train_df.iloc[:, :-1].values
y_train = train_df.iloc[:, -1].values

X_test = test_df.iloc[:, :-1].values
y_test = test_df.iloc[:, -1].values

# Reshape for CNN 1D: (samples, timesteps, channels)
X_train = X_train.reshape(-1, 187, 1)
X_test  = X_test.reshape(-1, 187, 1)

# One-hot encoding labels (5 classes)
y_train = to_categorical(y_train, num_classes=5)
y_test  = to_categorical(y_test, num_classes=5)

X_train.shape, y_train.shape

((87554, 187, 1), (87554, 5))

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

model = Sequential([
    Conv1D(32, kernel_size=5, activation='relu', input_shape=(187, 1)),
    MaxPooling1D(pool_size=2),

    Conv1D(64, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 183, 32)        │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 91, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 87, 64)         │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 43, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2752)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       176,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 187,013 (730.52 KB)

 Trainable params: 187,013 (730.52 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np

X = train_df.iloc[:, :-1].values
y = train_df.iloc[:, -1].values.astype(int)

X_tr, X_val, y_tr, y_val = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

X_tr  = X_tr.reshape(-1, 187, 1)
X_val = X_val.reshape(-1, 187, 1)

y_tr  = to_categorical(y_tr, 5)
y_val = to_categorical(y_val, 5)

In [ ]:
history = model.fit(
    X_tr, y_tr,
    epochs=5,
    batch_size=128,
    validation_data=(X_val, y_val),
    verbose=1
)

Epoch 1/5
306/616 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.8554 - loss: 0.5745

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

y_pred = model.predict(X_test, verbose=0)
y_pred_cls = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_true, y_pred_cls)
print("Confusion matrix:\n", cm)

print("\nClassification report:\n")
print(classification_report(y_true, y_pred_cls, digits=4))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)

plt.figure(figsize=(6,5))
plt.imshow(cm_norm)
plt.title("Normalized Confusion Matrix (Test)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.colorbar()
plt.xticks(range(5))
plt.yticks(range(5))

# Annotate
for i in range(5):
    for j in range(5):
        plt.text(j, i, f"{cm_norm[i, j]:.2f}",
                 ha="center", va="center", color="white")

plt.show()

In [ ]:
%cd /content
!rm -rf mlmed2026
!git clone https://github.com/hoangnghia1444/mlmed2026.git
%cd /content/mlmed2026
!ls


In [ ]:
!cp "/content/drive/MyDrive/Medical ML/Medical_ML.ipynb" /content/mlmed2026/Medical_ML.ipynb


In [ ]:
%%bash
cd /content/mlmed2026
sed -i 's/* Your name here/* Name: Hoang Nguyen Tuan Nghia/g' README.md
sed -i 's/* Your ID here/* Student ID: 23BI14335/g' README.md
sed -i 's/* Your team ID here/* Team: 4/g' README.md


In [ ]:
import getpass
TOKEN = getpass.getpass("Paste GitHub PAT (token): ")

In [ ]:
!cd /content/mlmed2026
!git status
!git config --global user.email "hoangtnghia2022@gmail.com"
!git config --global user.name "Tuan Nghia"
!git add README.md Medical_ML.ipynb
!git commit -m "Update README and add Medical ML notebook"

# đổi remote origin sang dạng có token (token sẽ không hiện lại nếu ông không print)
!git remote set-url origin https://{TOKEN}@github.com/hoangnghia1444/mlmed2026.git

# push
!git push origin main

!git push